# **Dask**

### Before we dive into dask, lets look at a simple usecase:
### **Let's say you want to figure out the best channels on Youtube to advertise a product on, across the globe**

In [ ]:
import pandas as pd
import os
DATA_DIR = os.path.join(os.getcwd(), "data")
COLS = [
    'video_id',
    'trending_date',
    'title',
    'channel_title',
    'category_id',
    'publish_time',
    'tags',
    'views',
    'likes',
    'dislikes',
    'comment_count',
    'thumbnail_link',
    'comments_disabled',
    'ratings_disabled',
    'video_error_or_removed'
]

df1 = pd.read_csv(os.path.join(DATA_DIR, "youtube", "USvideos.csv"), usecols=COLS)
df2 = pd.read_csv(os.path.join(DATA_DIR, "youtube", "GBvideos.csv"), usecols=COLS)

top_5_US_channels = df1.groupby("channel_title").sum().reset_index()[["channel_title", "views"]].sort_values(by="views", ascending=False).head(5)
top_5_UK_channels = df2.groupby("channel_title").sum().reset_index()[["channel_title", "views"]].sort_values(by="views", ascending=False).head(5)

df3 = pd.concat([df1, df2])
top_5_across_US_and_UK_channels = df3.groupby("channel_title").sum().reset_index()[["channel_title", "views"]].sort_values(by="views", ascending=False).head(5)

In [ ]:
top_5_across_US_and_UK_channels

### If we have n geographic regions, we have to create n dataframes - not really pretty

### We are also restricted by the size of the dataset that can fit in memory

# **So how does dask help exactly?**

### Dask = Dask Collections + Dask Task Graph

### Dask offers parallel versions for familiar data structures in pandas & numpy like

1. Dask Array (parallel numpy arrays)
2. Dask Bag (parallel python lists)
3. Dask Dataframe (parallel pandas dataframes)

### Dask also has few helpful parallelization primitives

- Dask Delayed (lazy parallelism)
- Dask Futures (real-time parallelism)

![](./img/dask-overview.svg)

### Let's see how we can solve our previous problem with dask

In [ ]:
import dask.dataframe as dd
import os
from distributed import Client
client = Client()
DATA_DIR = os.path.join(os.getcwd(), "data")

In [ ]:
df = dd.read_csv(
    os.path.join(DATA_DIR, "youtube", "*.csv"),
    encoding="latin1"
)

df.compute().size

In [ ]:
df.groupby("channel_title").sum().reset_index()[["channel_title", "views"]].nlargest(n=5, columns="views").compute()

### Lets look at how dask arrays are different from numpy arrays

In [ ]:
import dask.array as da
x = da.arange(100, chunks=20) # each chunk has 20
x

In [ ]:
x.npartitions, x.chunks, x.chunksize

In [ ]:
import numpy as np

for i in range(5):
    print(np.array(x.blocks[i]))

In [ ]:
x.map_blocks(lambda p: p + 2).compute() # map_blocks only accepts funcs which work element wise

### We have already seen dask dataframes but what exactly are dask bags?

In [ ]:
import dask.bag as db
y = db.from_sequence(range(10), npartitions=4)
y

In [ ]:
y.map(lambda x: x + 4).compute()

### Lets parallelize a simple operation, say parsing some CSV files

In [ ]:
import os
import csv

regions = ["CA", "DE", "FR", "GB", "IN", "JP", "KR", "MX", "RU", "US"]

def find_count_in_csv(region):
    csv_path = os.path.join("data", "youtube", f"{region}videos.csv")
    with open(csv_path, 'rt', encoding="latin1") as csvfile:
        rows = csv.reader(csvfile)
        return region, sum(1 for _ in rows)

tuples = db.from_sequence(regions).map(find_count_in_csv).compute()
dict(tuples)

### You can use delayed and futures to parallelize existing code

In [ ]:
from dask import delayed, compute
import time
from random import randrange

# @delayed
def func1(n):
    time.sleep(1)
    return n

# @delayed
def func2(n):
    time.sleep(2)
    return n ** 2

# @delayed
def func3(a, b):
    return a + b

def main_func(x, y):
    p = func1(x)
    q = func2(y)
    r = func3(p, q)
    return r

In [ ]:
%time main_func(4,5)
# %time main_func(4,5).compute()

In [ ]:
c1 = client.submit(func1, 4)
c2 = client.submit(func2, 5)
c3 = client.submit(func3, c1, c2)
%time c3.result()

### Dask futures are similar to Python 3+'s async/await and in fact extend it

In [ ]:
import dask.array as da
import numpy as np

def func(x):
    return np.tan(x) * np.arctan(x)

%time da.arange(10 ** 7).map_blocks(func, dtype=float).compute()

### Not all functions in numpy/pandas are supported by dask

- [Dask Array Scope](https://docs.dask.org/en/latest/array.html#scope)
- [Dask Bag Limitations](https://docs.dask.org/en/latest/bag.html#known-limitations)
- [Dask Dataframe Scope](https://docs.dask.org/en/latest/dataframe.html#scope)

# **Exercise**

Try parallelizing the python code you wrote earlier without using prange from numba. Do you notice any improvements?

Also see if you can find another implementation for "func".

**If you find any improvement, feel free to tweet about your experience with the handle @pyconfhyd**